In [ ]:
"""
To Make sure the Best Axon method works and also
to see if can decompose the axon any better 
(especially if can help with crosses and parts that aren't skeletonized             )

"""

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

In [5]:
import datajoint_utils as du
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-26 03:03:42,945 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-26 03:03:42,946 - settings - Setting database.user to celiib
INFO - 2021-02-26 03:03:42,947 - settings - Setting database.password to newceliipass
INFO - 2021-02-26 03:03:42,951 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-02-26 03:03:42,952 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-26 03:03:42,972 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-02-26 03:03:43,306 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-26 03:03:43,340 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-26 03:03:43,715 - settings - Setting enable_python_native_blobs to True


In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import networkx_utils as xu
import datajoint as dj
import neuron_visualizations as nviz
import neuron
import neuron_utils as nru
import proofreading_utils as pru
import classification_utils as clu
import system_utils as su
from importlib import reload
import trimesh_utils as tu

# Getting a more high fidelity axon skeleton

In [5]:
minnie.AutoProofreadValidationNeurons() & "axon_length>300"

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,proof_version the version of code used for this cell typing classification,multiplicity the total number of neurons that came from the parent segment id,cell_type_predicted morphology predicted by classifier,spine_category,n_axons Number of axon candidates identified,n_apicals Number of apicals identified,axon_length length (in um) of the classified axon skeleton,axon_area # area (in um^2) of the classified axon,nucleus_id id of nucleus from the flat segmentation Equivalent to Allen: 'id'.,nuclei_distance the distance to the closest nuclei (even if no matching nuclei found),n_nuclei_in_radius the number of nuclei within the search radius of 15000 belonging to that segment,n_nuclei_in_bbox the number of nuclei within the bounding box of that soma,"soma_x x coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_y y coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","soma_z z coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)",max_soma_n_faces The largest number of faces of the somas,max_soma_volume The largest volume of the somas the (volume in billions (10*9 nm^3)),n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,axon_angle_maximum the anlge of an identified axon,spine_density_classifier the number of spines divided by skeletal length for branches analyzed in classification,n_branches_processed the number branches used for the spine density analysis,skeletal_length_processed The total skeletal length of the viable branches used for the spine density analysis,n_branches_in_search_radius the number branches existing in the search radius used for spine density,skeletal_length_in_search_radius The total skeletal length of the branches existing in the search radius used for spine density,run_time the amount of time to run (seconds)
864691135113167769,0,0.25,30.00,5,0,0,2,1,excitatory,densely_spined,1,1,1521.4480575529685,1676.4575752748142,335175,917.6,1,1,198908,188769,19783

In [10]:
print("hi")

hi


In [11]:
multi_axon_segs,multi_axon_splits = (minnie.AutoProofreadNeurons2() & "cell_type_predicted='excitatory'"
 & "n_axons>1").fetch("segment_id","split_index")

In [18]:
segment_id,split_index = 864691135426651890,0 #long axon good for proofreading
#segment_id, split_index = 864691135446640468,0 #comes off the side of dendrite
segment_id,split_index = 864691135699269154,0
segment_id,split_index = 864691135589907979,0
segment_id, split_index = 864691135735426708,0
segment_id, split_index = 864691135645592260,0
segment_id, split_index= 864691135865518469,0

In [19]:
validation = True

In [20]:
du.plot_proofread_neuron(segment_id,
                        split_index=split_index,
                        plot_errored_faces=True,
                        plot_synapses=True,
                        plot_nuclei=True,
                        plot_axon=True,
                        validation=True)

Synapsse Results:
Total Number of Synapses = 5263
n_valid_syn_ids_presyn = 0
n_errored_syn_ids_presyn = 89
n_valid_syn_ids_postsyn = 5156
n_errored_syn_ids_postsyn = 18
Including axon mesh


/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [21]:
neuron_obj = du.decomposition_with_spine_recalculation(segment_id,
                                                      split_index=split_index)

Decompressing Neuron in minimal output mode...please wait
Pulled from Table Decomposition so setting split_index = 0
Dont need spine recalculation
Number of Neurons found = 1


In [22]:
if type(neuron_obj) == tuple:
    neuron_obj = neuron_obj[0][split_index]

In [23]:
neuron_obj.segment_id

864691135865518469

In [27]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")

KeyboardInterrupt: 

In [ ]:
clu.axon_classification(neuron_obj,
                        plot_candidates=False,
                       best_axon=True,
                       plot_axons=True)

# Applying the New Proofreading with High Fidelity Axon

In [33]:
output_dict= pru.proofread_neuron(neuron_obj,
                            plot_limb_branch_filter_with_disconnect_effect=True,
                            plot_final_filtered_neuron=False,
                            verbose=False)

filtered_neuron = output_dict["filtered_neuron"]
cell_type_info = output_dict["cell_type_info"]
filtering_info = output_dict["filtering_info"]

---- Part A: NOT Attempting to split neuron --------

--- Working on Neuron 0 ---


 ------ Part B: Axon Classification ---- 


base_final_widths = [263.70092017 201.69925104 397.83341247]
overall_ais_width = 263.7009201729347
ais_width = 287.74452789180344
Axon Classification = 46.93649220466614


 ------ Part C: Inhibitory Excitatory Classification ---- 


Did not pass final filters to continuing
Inhibitory Excitatory Classification = 18.8284432888031
cell type = excitatory


 ------ Part D: Neuron Filtering ---- 

 


 Using high_fidelity_axon_on_excitatory


---Step 1: Applying Dendrite on Axon Filtering

--- Working on filter 0:
function = dendrite_on_axon_merges
function __name__ = filter_away_dendrite_on_axon_merges
function arguments = {'use_pre_existing_axon_labels': True}





 ---- Total time for applying filter: 0.5024271011352539 -----


---- Step 2: Applying High Fidelity Axon -----
Starting axon branch was not the limb starting node so generating border vertices and comp

  0%|          | 0/1 [00:00<?, ?it/s]

root_curr = [1313789.   483423.7  759426.2]
Time for preparing soma vertices and root: 0.0005240440368652344
invalidation_d = 2000
cc_vertex_thresh = 10


100%|██████████| 1/1 [00:00<00:00, 14.54it/s]


meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 0.14470791816711426
connecting at the root
branches_touching_root = [0]
combining close nodes
combine_threshold = 1300


...


Decomposing first pass: 0.32952427864074707
Only Using MeshParty Skeletonization and Mesh Correspondence
Another print
Divinding into MP and MAP pieces: 0.0001609325408935547
Total time for MAP sublimb processing 6.9141387939453125e-06
---- Working on MP Decomposition #0 ----
Using Quicker soma_to_piece_touching_vertices because no MAP and only one sublimb_mesh piece 
MP filtering soma verts: 0.00018596649169921875
Fixing Possible Soma Extension Branch for Sublimb 0
Total time for mesh KDTree = 0.06955456733703613
sbv[0].reshape(-1,3) = [[1313789.   483423.7  759426.2]]
closest_sk_pt_coord BEFORE = [1312929.50303404  483380.11574483  759695.99395563]
current_skeleton.shape = (194, 2, 3)
node_for_stitch = 61: [1312929.50303404  483380.11574483  759695.99395563]
node_for_stitch AFTER = 61: [1312929.50303404  483380.11574483  759695.99395563]
possible_node_loc = [33 43]
possible_node_loc AFTER = [33 43]
curr_shortest_path = [61, 58.0, 56.0, 53.0, 50.0, 46.0, 42.0, 38.0, 33.0]
Current sti


2767 2768
For marked faces: None
empty_indices % = 0.0
 conflict_indices % = 0.0



AFTER face_lookup_resolved_test



Readjusting starting axon branch skeleton
Fixing the axon starting branch endpoint to align with upstream branch
Starting Branch 0 skeleton before fix: (194, 2, 3)
Starting Branch 0 skeleton AFTER fix: (195, 2, 3)
concept_network_dict = {0: [<networkx_utils.GraphOrderedEdges object at 0x7f353797e048>]}


----Working on seperate_graph 0----


---------Working on soma_idx = 0, soma_group_idx 0, endpt = [1313743.64179104  482405.25970149  760136.7119403 ]---------
Starting_edge inside branches_to_conept = [[1313743.64179104  482405.25970149  760136.7119403 ]
 [1313750.          482889.          760184.        ]]
At the start, starting_node (in terms of the skeleton, that shouldn't match the starting edge) = [8]
printing out current edge:
[[1313743.64179104  482405.25970149  760136.7119403 ]
 [1313750.          482889.          760184.        ]]
edge_endpoints_to_process was empty so exiting loop after 31 iterations
starting_node in concept map (that should match the starting edge) = 1
To


--- Working on filter 1:
function = crossovers
function __name__ = filter_away_crossovers
function arguments = {'axon_dependent': True, 'match_threshold': 50, 'require_two_pairs': False}



--- Working on filter 2:
function = low_branch_clusters
function __name__ = filter_away_low_branch_length_clusters
function arguments = {}



--- Working on filter 3:
function = double_back_and_width_change
function __name__ = filter_away_large_double_back_or_width_changes
function arguments = {'perform_double_back_errors': True, 'skip_double_back_errors_for_axon': False, 'width_jump_threshold': 250, 'running_width_jump_method': True, 'double_back_axon_like_threshold': 145, 'double_back_threshold': 120}



--- Working on filter 4:
function = high_degree_coordinates
function __name__ = filter_away_high_degree_coordinates
function arguments = {'axon_dependent': True, 'min_degree_to_find': 4}


min_degree_to_find = 4
min_degree_to_find = 4
min_degree_to_find = 4
min_degree_to_find = 4
min_degree_to_fi

In [34]:
nviz.visualize_neuron(filtered_neuron,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton
